In [ ]:
!pip install numpy==1.26.4 mediapipe opencv-python

In [ ]:
# video feed
cap = cv2.VideoCapture(0) #setting up video capture device 


with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
    while cap.isOpened():     #keep reading till the cam is open
        ret, frame = cap.read() # reading the current feed from cam
        frame = cv2.flip(frame, 1) 
        #recolour image to RGB
        image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable=False

        #make detection
        results=pose.process(image)

        # recolouring back to BGR
        image.flags.writeable=True
        image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #rendering detections 
        #we are passing the image , pose landmarks , connections of bodyparts
        mp_drawing.draw_landmarks(image ,results.pose_landmarks,mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),  # nodes colour 
                                mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)    # line colour 
                                 )

        print(results)
        
        cv2.imshow('Mediapipe Feed', image) #visualising the image from the frame
        
        if cv2.waitKey(10) & 0xFF == ord('q'): #if we close screen or press q we come out of loop
            break
        #0xff check which key we pressed 
    
    cap.release()
    cv2.destroyAllWindows() # closing all the video feed 

In [ ]:

import cv2
import mediapipe as mp
import numpy as np
import math
import time
import random
import pygame
import sys
from enum import Enum
from dataclasses import dataclass
from typing import List, Tuple, Optional

# Initialize Pygame for sound and enhanced graphics
pygame.mixer.init(frequency=22050, size=-16, channels=2, buffer=512)

class GameState(Enum):
    MENU = "menu"
    PLAYING = "playing"
    PAUSED = "paused"
    ROUND_END = "round_end"
    GAME_OVER = "game_over"

class MoveType(Enum):
    JAB = "JAB"
    UPPERCUT = "UPPERCUT"
    BLOCK = "BLOCK"
    SPECIAL = "SPECIAL"

@dataclass
class Particle:
    x: float
    y: float
    vx: float
    vy: float
    life: float
    max_life: float
    color: Tuple[int, int, int]
    size: int

@dataclass
class HitEffect:
    x: int
    y: int
    start_time: float
    duration: float
    effect_type: str

class RetroColors:
    # 8-bit inspired color palette
    NEON_CYAN = (0, 255, 255)
    NEON_MAGENTA = (255, 0, 255)
    NEON_GREEN = (0, 255, 128)
    NEON_YELLOW = (255, 255, 0)
    DANGER_RED = (255, 0, 64)
    ELECTRIC_BLUE = (64, 128, 255)
    GRID_PINK = (255, 0, 110)
    DARK_BG = (8, 8, 20)
    WHITE = (255, 255, 255)
    BLACK = (0, 0, 0)
    GRAY = (128, 128, 128)

class SoundManager:
    def __init__(self):
        self.sounds = {}
        self.load_sounds()

    def load_sounds(self):
        # Create simple sound effects using pygame
        try:
            # Generate simple tones for different effects
            self.generate_sound('hit', 440, 0.1)  # A4 note for hits
            self.generate_sound('combo', 880, 0.15)  # A5 for combos
            self.generate_sound('special', 660, 0.2)  # E5 for special moves
            self.generate_sound('round_start', 220, 0.3)  # A3 for round start
            self.generate_sound('victory', 330, 0.5)  # E4 for victory
        except:
            print("Audio system not available, continuing without sound...")

    def generate_sound(self, name: str, frequency: float, duration: float):
        sample_rate = 22050
        frames = int(duration * sample_rate)
        arr = np.zeros((frames, 2))

        for i in range(frames):
            wave = 4096 * np.sin(2 * np.pi * frequency * i / sample_rate)
            # Add envelope to prevent clicking
            envelope = min(i / (sample_rate * 0.01), 1.0, (frames - i) / (sample_rate * 0.01))
            arr[i] = [wave * envelope, wave * envelope]

        sound = pygame.sndarray.make_sound(arr.astype(np.int16))
        self.sounds[name] = sound

    def play(self, sound_name: str):
        if sound_name in self.sounds:
            try:
                self.sounds[sound_name].play()
            except:
                pass  # Ignore audio errors

class Player:
    def __init__(self, name: str, side: str, camera_id: int):
        self.name = name
        self.side = side  # 'left' or 'right'
        self.camera_id = camera_id  # Camera input index
        self.health = 100
        self.max_health = 100
        self.combo = 0
        self.score = 0
        self.last_move = ""
        self.move_time = 0
        self.is_blocking = False
        self.last_hit = 0
        self.invulnerable_until = 0
        self.special_meter = 0
        self.jab_stage = "Neutral"
        self.upper_stage = "Neutral"

        # Position on screen
        self.x = 200 if side == 'left' else 1720
        self.y = 400

class RetroFightingGame:
    def __init__(self):
        # Initialize MediaPipe
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_pose = mp.solutions.pose

        # Game state
        self.state = GameState.MENU
        self.current_round = 1
        self.max_rounds = 3
        self.round_timer = 90.0
        self.round_start_time = 0
        self.game_start_time = time.time()

        # Players with separate cameras
        self.players = {
            'A': Player('FIGHTER A', 'left', 0),   # Camera 0 for Player A
            'B': Player('FIGHTER B', 'right', 1)   # Camera 1 for Player B
        }

        # Visual effects
        self.particles: List[Particle] = []
        self.hit_effects: List[HitEffect] = []
        self.screen_shake = 0
        self.screen_shake_decay = 0.9

        # Background animation
        self.grid_offset = 0
        self.time_accumulator = 0

        # Sound system
        self.sound_manager = SoundManager()

        # Damage values
        self.damage_values = {
            MoveType.JAB: 8,
            MoveType.UPPERCUT: 12,
            MoveType.SPECIAL: 20
        }

        # Camera feed settings - BIGGER FEEDS
        self.camera_feed_width = 480   # Increased from 300
        self.camera_feed_height = 320  # Increased from 200
        self.show_pose_landmarks = True

        # Initialize dual cameras
        self.init_cameras()

    def init_cameras(self):
        """Initialize separate cameras for each player"""
        self.cameras = {}

        # Try to initialize both cameras
        print("Initializing cameras...")

        # Camera for Player A (left)
        cap_a = cv2.VideoCapture(0)
        if cap_a.isOpened():
            self.cameras['A'] = cap_a
            print("✓ Player A camera (Camera 0) initialized successfully")
        else:
            print("✗ Warning: Could not initialize Player A camera (Camera 0)")
            self.cameras['A'] = None

        # Camera for Player B (right)
        cap_b = cv2.VideoCapture(1)
        if cap_b.isOpened():
            self.cameras['B'] = cap_b
            print("✓ Player B camera (Camera 1) initialized successfully")
        else:
            print("✗ Warning: Could not initialize Player B camera (Camera 1)")
            print("  Falling back to shared camera for Player B")
            self.cameras['B'] = None

        # If we couldn't get two cameras, use one camera for both (fallback)
        if self.cameras['A'] is None and self.cameras['B'] is None:
            print("Error: No cameras available!")
            sys.exit(1)
        elif self.cameras['B'] is None and self.cameras['A'] is not None:
            # Use camera A for both players as fallback
            self.cameras['B'] = self.cameras['A']
            print("  Using Camera 0 for both players as fallback")

    def calculate_angle(self, a, b, c):
        """Calculate angle between three points"""
        a = np.array([a.x, a.y])
        b = np.array([b.x, b.y])
        c = np.array([c.x, c.y])

        radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
        angle = np.abs(radians * 180.0 / np.pi)

        if angle > 180:
            angle = 360 - angle

        return angle

    def euclidean_distance(self, a, b):
        """Calculate distance between two points"""
        return math.sqrt((a.x - b.x)**2 + (a.y - b.y)**2)

    def detect_moves(self, landmarks, player_id: str):
        """Detect player moves from pose landmarks"""
        if not landmarks:
            return

        player = self.players[player_id]
        lm = landmarks.landmark

        current_time = time.time()

        # Left hand JAB detection
        l_sh = lm[self.mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        l_el = lm[self.mp_pose.PoseLandmark.LEFT_ELBOW.value]
        l_wr = lm[self.mp_pose.PoseLandmark.LEFT_WRIST.value]

        if l_el.visibility > 0.5 and l_wr.visibility > 0.5:
            dist_lw_nose = self.euclidean_distance(l_wr, lm[self.mp_pose.PoseLandmark.NOSE.value])
            angle_l = self.calculate_angle(l_sh, l_el, l_wr)

            stage = "Extended" if angle_l > 160 and dist_lw_nose > 0.25 else "Neutral"

            if player.jab_stage == "Extended" and stage == "Neutral":
                self.execute_move(player_id, MoveType.JAB)

            player.jab_stage = stage

        # Right hand UPPERCUT detection
        r_sh = lm[self.mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        r_el = lm[self.mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        r_wr = lm[self.mp_pose.PoseLandmark.RIGHT_WRIST.value]

        if r_el.visibility > 0.5 and r_wr.visibility > 0.5:
            if r_wr.y > r_sh.y + 0.15:
                stage = "Down"
            elif r_wr.y < r_sh.y - 0.08:
                stage = "Extended"
            else:
                stage = "Neutral"

            if player.upper_stage == "Extended" and stage == "Neutral":
                self.execute_move(player_id, MoveType.UPPERCUT)

            player.upper_stage = stage

        # Block detection (arms crossed)
        if (l_wr.x < r_wr.x and l_el.visibility > 0.5 and r_el.visibility > 0.5 and
            abs(l_wr.y - r_wr.y) < 0.1 and l_wr.y < l_sh.y):
            player.is_blocking = True
        else:
            player.is_blocking = False

    def execute_move(self, player_id: str, move_type: MoveType):
        """Execute a detected move"""
        attacker = self.players[player_id]
        defender = self.players['B' if player_id == 'A' else 'A']

        current_time = time.time()

        # Check if player can attack (not in invulnerable state)
        if current_time < attacker.invulnerable_until:
            return

        # Update attacker
        attacker.last_move = move_type.value
        attacker.move_time = current_time
        attacker.combo += 1
        attacker.special_meter = min(100, attacker.special_meter + 10)

        # Calculate damage based on combo
        base_damage = self.damage_values[move_type]
        combo_multiplier = 1.0 + (attacker.combo - 1) * 0.2
        total_damage = int(base_damage * combo_multiplier)

        # Check if defender is blocking
        damage_dealt = total_damage
        if defender.is_blocking:
            damage_dealt = max(1, total_damage // 3)  # Reduced damage when blocking
            self.create_particles(defender.x, defender.y, RetroColors.ELECTRIC_BLUE, 5)
        else:
            # Full damage, create hit effects
            self.create_particles(defender.x, defender.y, RetroColors.DANGER_RED, 15)
            self.screen_shake = 10

        # Apply damage
        if current_time > defender.invulnerable_until:
            defender.health = max(0, defender.health - damage_dealt)
            defender.last_hit = current_time
            defender.invulnerable_until = current_time + 0.3  # 300ms invulnerability
            defender.combo = 0  # Reset defender combo

        # Create hit effect
        effect_type = 'block' if defender.is_blocking else 'hit'
        self.hit_effects.append(HitEffect(
            x=defender.x,
            y=defender.y,
            start_time=current_time,
            duration=0.3,
            effect_type=effect_type
        ))

        # Play sound
        sound_name = 'combo' if attacker.combo > 3 else 'hit'
        self.sound_manager.play(sound_name)

        # Check for round end
        if defender.health <= 0:
            self.end_round(player_id)

    def create_particles(self, x: int, y: int, color: Tuple[int, int, int], count: int):
        """Create particle explosion effect"""
        for _ in range(count):
            angle = random.uniform(0, 2 * math.pi)
            speed = random.uniform(2, 8)

            particle = Particle(
                x=x + random.randint(-20, 20),
                y=y + random.randint(-20, 20),
                vx=math.cos(angle) * speed,
                vy=math.sin(angle) * speed,
                life=1.0,
                max_life=1.0,
                color=color,
                size=random.randint(2, 6)
            )
            self.particles.append(particle)

    def update_particles(self, dt: float):
        """Update particle systems"""
        for particle in self.particles[:]:
            particle.x += particle.vx * dt * 60
            particle.y += particle.vy * dt * 60
            particle.vy += 200 * dt  # Gravity
            particle.life -= dt * 2

            if particle.life <= 0:
                self.particles.remove(particle)

        # Update hit effects
        current_time = time.time()
        self.hit_effects = [effect for effect in self.hit_effects 
                           if current_time - effect.start_time < effect.duration]

        # Update screen shake
        self.screen_shake *= self.screen_shake_decay
        if self.screen_shake < 0.1:
            self.screen_shake = 0

    def draw_camera_feed(self, frame, camera_frame, x: int, y: int, player_name: str, 
                        pose_results=None, is_left_player=True):
        """Draw camera feed in specified position with pose landmarks"""
        if camera_frame is None:
            # Draw "NO CAMERA" placeholder
            placeholder = np.zeros((self.camera_feed_height, self.camera_feed_width, 3), dtype=np.uint8)
            self.draw_retro_text(placeholder, "NO CAMERA", 
                               self.camera_feed_width//2 - 60, self.camera_feed_height//2, 
                               RetroColors.DANGER_RED, 0.8)
            feed = placeholder
        else:
            # Resize camera feed to larger size
            feed = cv2.resize(camera_frame, (self.camera_feed_width, self.camera_feed_height))

            # Create a copy for pose landmarks
            if pose_results and pose_results.pose_landmarks and self.show_pose_landmarks:
                # Draw pose landmarks on the feed
                self.mp_drawing.draw_landmarks(
                    feed, pose_results.pose_landmarks, self.mp_pose.POSE_CONNECTIONS,
                    self.mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                    self.mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2)
                )

        # Add retro border effect (thicker for larger feed)
        border_color = RetroColors.NEON_CYAN if is_left_player else RetroColors.NEON_MAGENTA
        border_thickness = 4  # Thicker border for larger feeds
        cv2.rectangle(frame, (x-border_thickness, y-border_thickness), 
                     (x + self.camera_feed_width + border_thickness, 
                      y + self.camera_feed_height + border_thickness), 
                     border_color, border_thickness)

        # Add player label with retro styling (larger text)
        label_y = y - 20
        self.draw_retro_text(frame, player_name, x, label_y, border_color, 0.8)

        # Overlay the camera feed
        try:
            frame[y:y+self.camera_feed_height, x:x+self.camera_feed_width] = feed
        except:
            # Handle any dimension mismatch
            pass

        # Add corner decorations (larger for bigger feeds)
        corner_size = 15
        cv2.line(frame, (x, y), (x + corner_size, y), border_color, 3)
        cv2.line(frame, (x, y), (x, y + corner_size), border_color, 3)
        cv2.line(frame, (x + self.camera_feed_width, y), 
                (x + self.camera_feed_width - corner_size, y), border_color, 3)
        cv2.line(frame, (x + self.camera_feed_width, y), 
                (x + self.camera_feed_width, y + corner_size), border_color, 3)

    def draw_retro_background(self, frame):
        """Draw animated retro grid background"""
        height, width = frame.shape[:2]

        # Fill with dark background
        frame[:] = RetroColors.DARK_BG

        # Animated grid
        grid_size = 50
        self.grid_offset += 1

        # Draw horizontal lines
        for y in range(-grid_size, height + grid_size, grid_size):
            y_pos = (y + self.grid_offset) % (grid_size * 2)
            if y_pos < height:
                cv2.line(frame, (0, y_pos), (width, y_pos), RetroColors.GRID_PINK, 1)

        # Draw vertical lines
        for x in range(0, width, grid_size):
            cv2.line(frame, (x, 0), (x, height), RetroColors.GRID_PINK, 1)

        # Add scanline effect
        for y in range(0, height, 4):
            cv2.line(frame, (0, y), (width, y), (0, 0, 0), 1)

    def draw_pixel_health_bar(self, frame, x: int, y: int, health: int, max_health: int, 
                             color: Tuple[int, int, int], width: int = 300):
        """Draw pixel-style health bar"""
        bar_height = 20
        border_thickness = 2

        # Draw border
        cv2.rectangle(frame, 
                     (x - border_thickness, y - border_thickness), 
                     (x + width + border_thickness, y + bar_height + border_thickness),
                     RetroColors.WHITE, border_thickness)

        # Draw background
        cv2.rectangle(frame, (x, y), (x + width, y + bar_height), RetroColors.BLACK, -1)

        # Draw health segments
        segments = 10
        segment_width = width // segments
        filled_segments = int((health / max_health) * segments)

        for i in range(segments):
            seg_x = x + i * segment_width
            seg_color = color if i < filled_segments else RetroColors.GRAY

            cv2.rectangle(frame, 
                         (seg_x + 1, y + 1), 
                         (seg_x + segment_width - 1, y + bar_height - 1),
                         seg_color, -1)

        # Draw health text
        health_text = f"{health:3d}"
        cv2.putText(frame, health_text, (x + width + 10, y + 15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, RetroColors.WHITE, 2)

    def draw_retro_text(self, frame, text: str, x: int, y: int, 
                       color: Tuple[int, int, int], size: float = 1.0):
        """Draw pixelated text with outline"""
        thickness = max(1, int(size * 2))

        # Draw outline
        for dx in [-1, 0, 1]:
            for dy in [-1, 0, 1]:
                if dx != 0 or dy != 0:
                    cv2.putText(frame, text, (x + dx, y + dy), 
                               cv2.FONT_HERSHEY_SIMPLEX, size, 
                               RetroColors.BLACK, thickness)

        # Draw main text
        cv2.putText(frame, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, size, color, thickness)

    def draw_game_hud(self, frame):
        """Draw game HUD elements - adjusted for larger camera feeds"""
        height, width = frame.shape[:2]

        # Player A (left side) - moved more inward to avoid larger camera feed
        player_a = self.players['A']
        hud_left_x = self.camera_feed_width + 30  # Start after larger camera feed
        self.draw_retro_text(frame, "PLAYER A", hud_left_x, 50, RetroColors.NEON_CYAN, 0.8)
        self.draw_pixel_health_bar(frame, hud_left_x, 70, player_a.health, player_a.max_health, 
                                  RetroColors.NEON_GREEN, 200)  # Shorter width for larger feeds

        # Player B (right side) - moved more inward to avoid larger camera feed
        player_b = self.players['B']
        hud_right_x = width - self.camera_feed_width - 230  # End before larger camera feed
        self.draw_retro_text(frame, "PLAYER B", hud_right_x, 50, RetroColors.NEON_MAGENTA, 0.8)
        self.draw_pixel_health_bar(frame, hud_right_x, 70, player_b.health, player_b.max_health,
                                  RetroColors.NEON_GREEN, 200)  # Shorter width for larger feeds

        # Round and timer info (center)
        center_x = width // 2
        self.draw_retro_text(frame, f"ROUND {self.current_round}/{self.max_rounds}", 
                           center_x - 80, 50, RetroColors.NEON_YELLOW, 1.0)

        if self.state == GameState.PLAYING:
            elapsed = time.time() - self.round_start_time
            remaining = max(0, self.round_timer - elapsed)
            self.draw_retro_text(frame, f"TIME: {int(remaining):02d}", 
                               center_x - 60, 100, RetroColors.WHITE, 0.8)

        # Show moves and combos
        for player_id, player in self.players.items():
            if player_id == 'A':
                x_pos = hud_left_x
            else:
                x_pos = hud_right_x

            if player.last_move and time.time() - player.move_time < 1.0:
                move_color = RetroColors.NEON_GREEN if player.combo > 1 else RetroColors.WHITE
                self.draw_retro_text(frame, f"{player.last_move}", x_pos, 150, move_color, 0.7)

                if player.combo > 1:
                    self.draw_retro_text(frame, f"COMBO x{player.combo}!", 
                                       x_pos, 180, RetroColors.NEON_YELLOW, 0.6)

            # Special meter
            if player.special_meter > 0:
                meter_width = int(100 * (player.special_meter / 100))
                cv2.rectangle(frame, (x_pos, 200), (x_pos + meter_width, 210), 
                             RetroColors.NEON_CYAN, -1)
                cv2.rectangle(frame, (x_pos, 200), (x_pos + 100, 210), 
                             RetroColors.WHITE, 1)

        # Draw particles
        for particle in self.particles:
            if particle.life > 0:
                alpha = particle.life / particle.max_life
                size = int(particle.size * alpha)
                if size > 0:
                    cv2.circle(frame, (int(particle.x), int(particle.y)), size, 
                             particle.color, -1)

        # Draw hit effects
        current_time = time.time()
        for effect in self.hit_effects:
            progress = (current_time - effect.start_time) / effect.duration
            if progress < 1.0:
                if effect.effect_type == 'hit':
                    size = int(50 * (1 - progress))
                    cv2.circle(frame, (effect.x, effect.y), size, 
                             RetroColors.DANGER_RED, 3)
                    self.draw_retro_text(frame, "POW!", effect.x - 30, effect.y, 
                                       RetroColors.NEON_YELLOW, 1.0)
                elif effect.effect_type == 'block':
                    cv2.rectangle(frame, (effect.x - 30, effect.y - 30), 
                                (effect.x + 30, effect.y + 30), 
                                RetroColors.ELECTRIC_BLUE, 3)
                    self.draw_retro_text(frame, "BLOCK!", effect.x - 40, effect.y, 
                                       RetroColors.ELECTRIC_BLUE, 0.8)

    def draw_menu(self, frame):
        """Draw main menu"""
        height, width = frame.shape[:2]

        # Title
        title_y = height // 4
        self.draw_retro_text(frame, "RETRO FIGHT CLUB", width//2 - 200, title_y, 
                           RetroColors.NEON_CYAN, 2.0)

        # Subtitle
        self.draw_retro_text(frame, "DUAL CAMERA POSE DETECTION FIGHTING", width//2 - 250, title_y + 60, 
                           RetroColors.NEON_MAGENTA, 1.0)

        # Instructions
        instructions = [
            "DUAL CAMERA SETUP:",
            "Player A: Left camera feed (Camera 0)",
            "Player B: Right camera feed (Camera 1)",
            "",
            "CONTROLS:",
            "Left Hand Extension = JAB",
            "Right Hand Up/Down = UPPERCUT", 
            "Arms Crossed = BLOCK",
            "",
            "Press SPACE to start",
            "Press C to toggle pose landmarks",
            "Press ESC to quit"
        ]

        start_y = height // 2 - 80
        for i, instruction in enumerate(instructions):
            if "DUAL CAMERA" in instruction:
                color = RetroColors.NEON_YELLOW
            elif "Press" in instruction:
                color = RetroColors.NEON_YELLOW
            elif "Player A" in instruction or "Player B" in instruction:
                color = RetroColors.NEON_GREEN
            else:
                color = RetroColors.WHITE
            self.draw_retro_text(frame, instruction, width//2 - 200, start_y + i * 30, 
                               color, 0.6)

    def draw_round_end(self, frame):
        """Draw round end screen"""
        height, width = frame.shape[:2]

        winner = max(self.players.values(), key=lambda p: p.score)

        self.draw_retro_text(frame, f"ROUND {self.current_round} COMPLETE!", 
                           width//2 - 150, height//2 - 50, RetroColors.NEON_YELLOW, 1.5)

        self.draw_retro_text(frame, f"{winner.name} WINS!", 
                           width//2 - 80, height//2, RetroColors.NEON_GREEN, 1.2)

        self.draw_retro_text(frame, "Press SPACE for next round", 
                           width//2 - 120, height//2 + 50, RetroColors.WHITE, 0.8)

    def draw_game_over(self, frame):
        """Draw game over screen"""
        height, width = frame.shape[:2]

        winner = max(self.players.values(), key=lambda p: p.score)

        self.draw_retro_text(frame, "GAME OVER!", width//2 - 80, height//2 - 100, 
                           RetroColors.DANGER_RED, 2.0)

        self.draw_retro_text(frame, f"CHAMPION: {winner.name}!", width//2 - 120, height//2 - 50, 
                           RetroColors.NEON_YELLOW, 1.5)

        self.draw_retro_text(frame, f"Final Score: {winner.score}-{(3-winner.score)}", 
                           width//2 - 80, height//2, RetroColors.WHITE, 1.0)

        self.draw_retro_text(frame, "Press R to restart", width//2 - 80, height//2 + 50, 
                           RetroColors.NEON_CYAN, 0.8)
        self.draw_retro_text(frame, "Press ESC to quit", width//2 - 80, height//2 + 80, 
                           RetroColors.WHITE, 0.8)

    def start_round(self):
        """Start a new round"""
        self.state = GameState.PLAYING
        self.round_start_time = time.time()

        # Reset player health and states
        for player in self.players.values():
            player.health = player.max_health
            player.combo = 0
            player.last_move = ""
            player.is_blocking = False
            player.last_hit = 0
            player.invulnerable_until = 0

        self.sound_manager.play('round_start')

    def end_round(self, winner_id: str):
        """End current round"""
        self.players[winner_id].score += 1

        if self.players[winner_id].score >= (self.max_rounds // 2) + 1:
            # Game over
            self.state = GameState.GAME_OVER
            self.sound_manager.play('victory')
        else:
            # Next round
            self.state = GameState.ROUND_END
            self.current_round += 1

    def reset_game(self):
        """Reset game to initial state"""
        self.state = GameState.MENU
        self.current_round = 1

        for player in self.players.values():
            player.health = player.max_health
            player.score = 0
            player.combo = 0
            player.last_move = ""
            player.is_blocking = False
            player.last_hit = 0
            player.invulnerable_until = 0
            player.special_meter = 0

    def handle_input(self, key):
        """Handle keyboard input"""
        if key == ord(' '):  # Space
            if self.state == GameState.MENU:
                self.current_round = 1
                self.start_round()
            elif self.state == GameState.ROUND_END:
                self.start_round()
        elif key == ord('r') or key == ord('R'):
            if self.state == GameState.GAME_OVER:
                self.reset_game()
        elif key == ord('c') or key == ord('C'):
            # Toggle camera feed visibility
            self.show_pose_landmarks = not self.show_pose_landmarks
            print(f"Pose landmarks: {'ON' if self.show_pose_landmarks else 'OFF'}")
        elif key == 27:  # ESC
            return False
        elif key == ord('p') or key == ord('P'):
            if self.state == GameState.PLAYING:
                self.state = GameState.PAUSED
            elif self.state == GameState.PAUSED:
                self.state = GameState.PLAYING

        return True

    def capture_camera_frames(self):
        """Capture frames from both cameras"""
        frames = {}

        for player_id in ['A', 'B']:
            camera = self.cameras.get(player_id)
            if camera is not None:
                ret, frame = camera.read()
                if ret:
                    frames[player_id] = cv2.flip(frame, 1)  # Flip horizontally
                else:
                    frames[player_id] = None
            else:
                frames[player_id] = None

        return frames

    def run(self):
        """Main game loop"""
        print("Starting Retro Fighting Game with Dual Camera Feeds...")
        print("Camera Setup:")
        print("- Player A: Camera 0 (left side)")
        print("- Player B: Camera 1 (right side)")
        print("- Larger camera feeds: 480x320 pixels")
        print("Press 'C' to toggle pose landmarks on/off")

        # Initialize pose detection for both cameras
        pose_a = self.mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.5)
        pose_b = self.mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.5)

        last_time = time.time()

        try:
            while True:
                current_time = time.time()
                dt = current_time - last_time
                last_time = current_time

                # Capture frames from both cameras
                camera_frames = self.capture_camera_frames()

                # Create main display frame
                main_frame = np.zeros((1080, 1920, 3), dtype=np.uint8)

                # Apply screen shake to main frame
                if self.screen_shake > 0:
                    shake_x = int(random.uniform(-self.screen_shake, self.screen_shake))
                    shake_y = int(random.uniform(-self.screen_shake, self.screen_shake))

                    height, width = main_frame.shape[:2]
                    M = np.float32([[1, 0, shake_x], [0, 1, shake_y]])
                    main_frame = cv2.warpAffine(main_frame, M, (width, height))

                # Process pose detection for both players only during gameplay
                pose_results = {'A': None, 'B': None}

                if self.state == GameState.PLAYING:
                    # Process Player A
                    if camera_frames['A'] is not None:
                        image_a = cv2.cvtColor(camera_frames['A'], cv2.COLOR_BGR2RGB)
                        image_a.flags.writeable = False
                        pose_results['A'] = pose_a.process(image_a)

                        if pose_results['A'].pose_landmarks:
                            self.detect_moves(pose_results['A'].pose_landmarks, 'A')

                    # Process Player B
                    if camera_frames['B'] is not None:
                        image_b = cv2.cvtColor(camera_frames['B'], cv2.COLOR_BGR2RGB)
                        image_b.flags.writeable = False
                        pose_results['B'] = pose_b.process(image_b)

                        if pose_results['B'].pose_landmarks:
                            self.detect_moves(pose_results['B'].pose_landmarks, 'B')

                    # Check for round timeout
                    if current_time - self.round_start_time > self.round_timer:
                        # Determine winner by health
                        if self.players['A'].health > self.players['B'].health:
                            self.end_round('A')
                        elif self.players['B'].health > self.players['A'].health:
                            self.end_round('B')
                        else:
                            # Draw - continue round
                            self.round_start_time = current_time

                # Draw background and UI based on state
                self.draw_retro_background(main_frame)

                # Show larger camera feeds (except in menu)
                if self.state != GameState.MENU:
                    # Left camera feed for Player A (larger size)
                    self.draw_camera_feed(main_frame, camera_frames['A'], 20, 200, 
                                        "PLAYER A CAMERA", pose_results['A'], is_left_player=True)

                    # Right camera feed for Player B (larger size) 
                    self.draw_camera_feed(main_frame, camera_frames['B'], 
                                        1920 - self.camera_feed_width - 20, 200, 
                                        "PLAYER B CAMERA", pose_results['B'], is_left_player=False)

                # Draw UI based on state
                if self.state == GameState.MENU:
                    self.draw_menu(main_frame)
                elif self.state == GameState.PLAYING:
                    self.draw_game_hud(main_frame)
                elif self.state == GameState.PAUSED:
                    self.draw_game_hud(main_frame)
                    height, width = main_frame.shape[:2]
                    self.draw_retro_text(main_frame, "PAUSED", width//2 - 50, height//2, 
                                       RetroColors.NEON_YELLOW, 2.0)
                elif self.state == GameState.ROUND_END:
                    self.draw_round_end(main_frame)
                elif self.state == GameState.GAME_OVER:
                    self.draw_game_over(main_frame)

                # Update effects
                self.update_particles(dt)

                # Display frame
                cv2.imshow("Retro Fight Club - Dual Camera", main_frame)

                # Handle input
                key = cv2.waitKey(1) & 0xFF
                if key != 255:  # Key was pressed
                    if not self.handle_input(key):
                        break

        except KeyboardInterrupt:
            print("\nGame interrupted by user")
        finally:
            # Cleanup
            for camera in self.cameras.values():
                if camera is not None:
                    camera.release()
            cv2.destroyAllWindows()
            pygame.mixer.quit()
            pose_a.close()
            pose_b.close()

if __name__ == "__main__":
    try:
        game = RetroFightingGame()
        game.run()
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure you have cameras connected and the required libraries installed:")
        print("pip install opencv-python mediapipe pygame numpy")
        print("\nCamera requirements:")
        print("- Camera 0: For Player A (left)")
        print("- Camera 1: For Player B (right)")
